In [1]:
import glob
import os.path
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
from sklearn.model_selection import KFold

In [2]:
#Inception-V3模型瓶颈层的节点个数
BOTTLENECK_TENSOR_SIZE = 2048

# Inception-V3 模型中代表瓶颈层结果的张量名称。在谷歌提供的Inception-v3模型中，这个张量名称是
# 'pool_3/_reshape:0'。在训练的模型时，可以通过tensor.name来获取张量的名称
BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'

# 图像输入张量所对应的名称
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'

# 下载的谷歌训练好的Inception-v3模型文件目录
MODEL_DIR = 'model'

# 下载的谷歌训练好的Inception-v3模型文件名
MODEL_FILE = 'tensorflow_inception_graph.pb'

# 因为一个训练数据会被多次使用，所以可以将原始图像通过Inception-v3模型计算得到
# 的特征向量保存在文件中，免去重复的计算。下面的变量定义了这些文件的存放地址
CACHE_DIR = 'bottleneck-15'

# 图片数据文件夹。在这个文件夹中每一个文件夹代表一个需要区分的类别，每个子文件夹
# 中存放了对应类别的图片
INPUT_DATA = 'data\\img_CA'

# 交叉验证的数值
KFOLD = 5

# 定义神经网络的设置
LEARNING_RATE = 0.001
STEPS = 4000
BATCH = 100

'''这个函数从数据文件夹中读取所有的图片列表并按训练、测试数据分开'''
def create_image_lists():
    # 得到的所有图片都存放在result这个字典里。这个字典的key为类别的名称，
    # value也是一个字典，字典里存储了所有的图片名称
    results = {}
    # 获取当前目录下所有的子目录
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    # 得到的第一个目录是当前目录，不需要考虑
    is_root_dir = True
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue

        # 获取当前目录下所有的有效图片文件
        extensions = ['jpg']
        file_list = []
        dir_name = os.path.basename(sub_dir)
        for extension in extensions:
            file_glob = os.path.join(INPUT_DATA, dir_name, '*.'+extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list: continue
            
        # 当前目录下有效图片文件的数量
        N = len(file_list)
        
        # 通过目录名获取类别的名称
        label_name = dir_name.lower()
        # 初始化当前类别的训练数据集、测试数据集和验证数据集
        #training_images = [] #二维列表，共KFOLD行，每行是一个列表，
        #testing_images = [] #二维列表，同上
        imgs = [] # 当前目录下所有的图片名称列表
        for file_name in file_list:
            base_name = os.path.basename(file_name)
            imgs.append(base_name)
            
        # 将当前类别的数据放入结果字典
        results[label_name] = {
            'dir': dir_name,
            'datasets': imgs
        }
   
      
    # 返回整理好的所有数据
    kf = KFold(n_splits = KFOLD,shuffle=True)
    results_list = {}
    
    
    for key in results.keys():
        ind = range(len(results[key]['datasets']))
        training = []
        testing = []
        d = results[key]['datasets']

        for train, test in kf.split(ind):
            training.append( [d[i] for i in train] )
            testing.append( [d[i] for i in test])
        
        results_list[key]={
            'dir': results[key]['dir'],
            'training': training,
            'testing': testing
    }
    return results_list


'''
这个函数通过类别名称、所属数据集和图片编号获取一张图片的地址。
image_lists 参数给出了所有图片信息
image_dir 参数给出了根目录。存放图片数据的根目录和存放图片特征向量的根目录的根目录地址不同。
label_name 参数给定了类别的名称
index 参数给定了需要获取的图片的编号
category 参数指定了需要获取的图片是在训练数据集、测试数据集还是验证数据集
'''
def get_image_path(image_lists, image_dir, label_name, index, category):
    # 获取给定类别中所有图片的信息
    label_lists = image_lists[label_name]
    # 根据所属数据集的名称获取集合中的全部图片信息
    category_list = label_lists[category]
    mod_index = index % len(category_list)
    # 获取图片的文件名
    base_name = category_list[mod_index]
    sub_dir = label_lists['dir']
    # 最终的地址为数据根目录的地址加上类别的文件夹加上图片的名称
    full_path = os.path.join(image_dir, sub_dir, base_name)
    return full_path


'''# 这个函数通过类别名称、所属数据集和图片编号获取经过Inception-v3模型处理之后的特征向量文件特征'''
def get_bottleneck_path(image_lists, label_name, index, category):
    return get_image_path(image_lists, CACHE_DIR, label_name, index, category) + '.txt'


'''# 这个函数使用加载的训练好的Inception-v3模型处理一张图片，得到这个图片的特征向量'''
def run_bottleneck_on_image(sess, image_data, image_data_tensor, bottleneck_tensor):
    # 这个过程实践上就是将当前图片作为输入计算瓶颈张量的值。这个瓶颈张量的值就是
    # 这张图片的新的特征向量
    bottleneck_values = sess.run(bottleneck_tensor, {image_data_tensor: image_data})
    # 经过卷积神经网络处理的结果是一个四维数组需要将这个结果压缩成一个特征
    # 向量（一维数组）
    bottleneck_values = np.squeeze(bottleneck_values)
    return bottleneck_values


'''# 这个函数获取一张图片经过Inception-v3模型处理之后的特征向量。这个函数会先试图寻找
# 已经计算且保存下来的特征向量，如果找不到则先计算这个特征向量，然后保存到文件'''
def get_or_create_bottleneck(sess, image_lists, label_name, index, category,
                             jpeg_data_tensor, bottleneck_tensor):
    # 获取一张图片对应的特征向量文件的路径
    label_lists = image_lists[label_name]
    sub_dir = label_lists['dir']
    sub_dir_path = os.path.join(CACHE_DIR, sub_dir)
    if not os.path.exists(sub_dir_path): os.makedirs(sub_dir_path)
    bottleneck_path = get_bottleneck_path(image_lists, label_name, index, category)

    # 如果这个特征向量文件不存在，则通过Inception-v3模型来计算特征向量，并将计算
    # 的结果存入文件
    if not os.path.exists(bottleneck_path):
        # 获取原始的图片路径
        image_path = get_image_path(image_lists, INPUT_DATA, label_name, index, category)
        # 获取图片内容
        image_data = gfile.FastGFile(image_path, 'rb').read()
        # 通过Inception-v3模型计算特征向量
        bottleneck_values = run_bottleneck_on_image(sess, image_data, jpeg_data_tensor, bottleneck_tensor)
        # 将计算得到的特征向量存入文件
        bottleneck_string = ','.join(str(x) for x in bottleneck_values)
        with open(bottleneck_path, 'w') as bottleneck_file:
            bottleneck_file.write(bottleneck_string)
    else:
        # 直接从文件中获取图片相应的特征向量
        with open(bottleneck_path, 'r') as bottleneck_file:
            bottleneck_string = bottleneck_file.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
    # 返回特征向量
    return bottleneck_values


'''# 这个函数随机获取一个batch的图片作为训练数据'''
def get_random_cached_bottlenecks(sess, n_classes, image_lists, how_many, category, jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    for _ in range(how_many):
        # 随机一个类别和图片的编号加入当前的训练数据
        label_index = random.randrange(n_classes)
        label_name = list(image_lists.keys())[label_index]
        image_index = random.randrange(65536)
        bottleneck = get_or_create_bottleneck(
            sess, image_lists, label_name, image_index, category, jpeg_data_tensor, bottleneck_tensor)
        ground_truth = np.zeros(n_classes, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)

    return bottlenecks, ground_truths


'''# 这个函数获取全部的测试数据。在最终测试的时候需要在所有的测试数据上计算正确率'''
def get_test_bottlenecks(sess, image_lists, n_classes, jpeg_data_tensor, bottleneck_tensor):
    bottlenecks = []
    ground_truths = []
    label_name_list = list(image_lists.keys())
    # 枚举所有的类别和每个类别中的测试图片
    for label_index, label_name in enumerate(label_name_list):
        category = 'testing'
        for index, unused_base_name in enumerate(image_lists[label_name][category]):
            # 通过Inception-v3模型计算图片对应的特征向量，并将其加入最终数据的列表
            bottleneck = get_or_create_bottleneck(sess, image_lists, label_name, index, category,jpeg_data_tensor, bottleneck_tensor)
            ground_truth = np.zeros(n_classes, dtype=np.float32)
            ground_truth[label_index] = 1.0
            bottlenecks.append(bottleneck)
            ground_truths.append(ground_truth)
    return bottlenecks, ground_truths


def main():
    # 读取所有图片
    images = create_image_lists()
    n_classes = len(images.keys())

    # 读取已经训练好的Inception-v3模型。
    with gfile.FastGFile(os.path.join(MODEL_DIR, MODEL_FILE), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # 加载读取的Inception-v3模型，并返回数据输入所对应的张量以及计算瓶颈
    # 层结果所对应的张量
    bottleneck_tensor, jpeg_data_tensor = tf.import_graph_def(
        graph_def, return_elements=[BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME])

    # 定义新的神经网络输入，这个输入就是新的图片经过Inception-v3模型前向传播到达瓶颈层
    # 的节点取值。可以将这个过程类似的理解为一种特征提取
    bottleneck_input = tf.placeholder(tf.float32, [None, BOTTLENECK_TENSOR_SIZE], name='BottleneckInputPlaceholder')
    # 定义新的标准答案输入
    ground_truth_input = tf.placeholder(tf.float32, [None, n_classes], name='GroundTruthInput')

    
    # 定义一层全链接层来解决新的图片分类问题。因为训练好的Inception-v3模型已经将原始
    # 图片抽象为更加容易分类的特征向量了，所以不需要再训练那么复杂的神经网络来完成
    # 这个新的分类任务
    with tf.name_scope('final_training_ops'):
        weights = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, n_classes], stddev=0.1))
        biases = tf.Variable(tf.zeros([n_classes]))
        logits = tf.matmul(bottleneck_input, weights) + biases
        final_tensor = tf.nn.softmax(logits)

    # 定义交叉熵损失函数。
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=ground_truth_input)
    #cross_entropy = tf.nn.weighted_cross_entropy_with_logits(logits=logits, targets=ground_truth_input, pos_weight=0.05)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    #train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy_mean)
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy_mean)

    
    
    # 计算正确率。
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(final_tensor, 1), tf.argmax(ground_truth_input, 1))
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        correct_number = 0
        total_number = 0
        # 共要进行KFOLD次训练和测试，
        for k in range(KFOLD):
            image_lists = {}
            testing_number = 0
            for key in images.keys():
                image_lists[key] = {
                    'dir': images[key]['dir'],
                    'training': images[key]['training'][k],
                    'testing': images[key]['testing'][k]
                }
                testing_number = testing_number + len( images[key]['testing'][k])
                
            total_number = total_number + testing_number
            
            print('%d K-Fold: training negative size %d; training positve size %d' % (k, len( images['negative']['training'][k]),
                                                                                          len( images['positive']['training'][k] )))
            
            print('%d K-Fold: tesing negative size %d; test positve size %d' % (k, len( images['negative']['testing'][k]), 
                                                                                    len( images['positive']['testing'][k] )))
            
            print('%d K-Fold: testing_number = %d, total_number = %d' % (k, testing_number, total_number))
            
            # 训练过程。
            for i in range(STEPS):
                # 每次获取一个batch的训练数据
                train_bottlenecks, train_ground_truth = get_random_cached_bottlenecks(
                    sess, n_classes, image_lists, BATCH, 'training', jpeg_data_tensor, bottleneck_tensor)
                sess.run(train_step,
                         feed_dict={bottleneck_input: train_bottlenecks, ground_truth_input: train_ground_truth})
                # 在测试数据上测试正确率
                
#                 if i % 100 == 0 or i + 1 == STEPS:
#                     validation_bottlenecks, validation_ground_truth = get_random_cached_bottlenecks(
#                         sess, n_classes, image_lists, BATCH, 'testing', jpeg_data_tensor, bottleneck_tensor)
#                     validation_accuracy = sess.run(evaluation_step, feed_dict={
#                         bottleneck_input: validation_bottlenecks, ground_truth_input: validation_ground_truth})
#                     print('K-Fold %d Step %d: testing accuracy on random sampled %d examples = %.1f%%' %
#                           (k,i, BATCH, validation_accuracy * 100))
                    
                    
                validation_bottlenecks, validation_ground_truth = get_random_cached_bottlenecks(
                    sess, n_classes, image_lists, BATCH, 'testing', jpeg_data_tensor, bottleneck_tensor)
            
                validation_accuracy = sess.run(evaluation_step, feed_dict={
                    bottleneck_input: validation_bottlenecks, ground_truth_input: validation_ground_truth})
                print('K-Fold %d Step %d: testing accuracy on random sampled %d examples = %.1f%%' %
                      (k,i, BATCH, validation_accuracy * 100))
              
            
                    
            # 在测试数据上测试正确率。
            test_bottlenecks, test_ground_truth = get_test_bottlenecks(
                sess, image_lists, n_classes, jpeg_data_tensor, bottleneck_tensor)
            test_accuracy = sess.run(evaluation_step, feed_dict={
                bottleneck_input: test_bottlenecks, ground_truth_input: test_ground_truth})
            
            correct_number = correct_number + test_accuracy*testing_number
            print('Final test accuracy = %.1f%%' % (test_accuracy * 100))
            print('correct_number = %d' % correct_number) 
            print('total_number = %d' % total_number)
            print('')
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('')
        print('') 
        print('') 
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~') 
        print('Final KFold test accuracy = %.1f%%' % (correct_number / 57348 * 100 ))

if __name__ == '__main__':
    main()

0 K-Fold: training negative size 42856; training positve size 3022
0 K-Fold: tesing negative size 10714; test positve size 756
0 K-Fold: testing_number = 11470, total_number = 11470
K-Fold 0 Step 0: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 1: testing accuracy on random sampled 100 examples = 40.0%
K-Fold 0 Step 2: testing accuracy on random sampled 100 examples = 39.0%
K-Fold 0 Step 3: testing accuracy on random sampled 100 examples = 43.0%
K-Fold 0 Step 4: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 5: testing accuracy on random sampled 100 examples = 46.0%
K-Fold 0 Step 6: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 7: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 8: testing accuracy on random sampled 100 examples = 48.0%
K-Fold 0 Step 9: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 10: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 0 Step 

K-Fold 0 Step 109: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 0 Step 110: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 111: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 0 Step 112: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 0 Step 113: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 0 Step 114: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 0 Step 115: testing accuracy on random sampled 100 examples = 44.0%
K-Fold 0 Step 116: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 0 Step 117: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 0 Step 118: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 119: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 0 Step 120: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 121: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 122: testin

K-Fold 0 Step 219: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 220: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 221: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 222: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 0 Step 223: testing accuracy on random sampled 100 examples = 47.0%
K-Fold 0 Step 224: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 0 Step 225: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 226: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 0 Step 227: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 0 Step 228: testing accuracy on random sampled 100 examples = 44.0%
K-Fold 0 Step 229: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 0 Step 230: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 231: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 232: testin

K-Fold 0 Step 329: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 0 Step 330: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 331: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 332: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 0 Step 333: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 0 Step 334: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 335: testing accuracy on random sampled 100 examples = 48.0%
K-Fold 0 Step 336: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 0 Step 337: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 0 Step 338: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 0 Step 339: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 340: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 341: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 342: testin

K-Fold 0 Step 439: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 0 Step 440: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 441: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 442: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 443: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 444: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 0 Step 445: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 446: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 0 Step 447: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 448: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 449: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 450: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 0 Step 451: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 452: testin

...

K-Fold 4 Step 3462: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 4 Step 3463: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3464: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 4 Step 3465: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3466: testing accuracy on random sampled 100 examples = 73.0%
K-Fold 4 Step 3467: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3468: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3469: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 4 Step 3470: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 4 Step 3471: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 4 Step 3472: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 4 Step 3473: testing accuracy on random sampled 100 examples = 70.0%
K-Fold 4 Step 3474: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Ste

K-Fold 4 Step 3570: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 4 Step 3571: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3572: testing accuracy on random sampled 100 examples = 72.0%
K-Fold 4 Step 3573: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3574: testing accuracy on random sampled 100 examples = 72.0%
K-Fold 4 Step 3575: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3576: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 4 Step 3577: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Step 3578: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 3579: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3580: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 4 Step 3581: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Step 3582: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Ste

K-Fold 4 Step 3678: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3679: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 4 Step 3680: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3681: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 4 Step 3682: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 4 Step 3683: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3684: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 4 Step 3685: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 4 Step 3686: testing accuracy on random sampled 100 examples = 69.0%
K-Fold 4 Step 3687: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Step 3688: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Step 3689: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3690: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 4 Ste

K-Fold 4 Step 3787: testing accuracy on random sampled 100 examples = 77.0%
K-Fold 4 Step 3788: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Step 3789: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 4 Step 3790: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 4 Step 3791: testing accuracy on random sampled 100 examples = 70.0%
K-Fold 4 Step 3792: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 4 Step 3793: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 4 Step 3794: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 3795: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 4 Step 3796: testing accuracy on random sampled 100 examples = 72.0%
K-Fold 4 Step 3797: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 4 Step 3798: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 3799: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 4 Ste

K-Fold 4 Step 3895: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 4 Step 3896: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3897: testing accuracy on random sampled 100 examples = 71.0%
K-Fold 4 Step 3898: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 4 Step 3899: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 4 Step 3900: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 4 Step 3901: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 4 Step 3902: testing accuracy on random sampled 100 examples = 69.0%
K-Fold 4 Step 3903: testing accuracy on random sampled 100 examples = 75.0%
K-Fold 4 Step 3904: testing accuracy on random sampled 100 examples = 74.0%
K-Fold 4 Step 3905: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 4 Step 3906: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 4 Step 3907: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 4 Ste

InternalError: Blas SGEMM launch failed : a.shape=(100, 2048), b.shape=(2048, 2), m=100, n=10, k=2048
    (_recv_Placeholder_0/_4, Variable/read)]]  

解决方法，重启电脑        

In [ ]:
0 K-Fold: training negative size 42856; training positve size 3022
0 K-Fold: tesing negative size 10714; test positve size 756
0 K-Fold: testing_number = 11470, total_number = 11470
K-Fold 0 Step 0: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 0 Step 1: testing accuracy on random sampled 100 examples = 47.0%
K-Fold 0 Step 2: testing accuracy on random sampled 100 examples = 45.0%
K-Fold 0 Step 3: testing accuracy on random sampled 100 examples = 46.0%
K-Fold 0 Step 4: testing accuracy on random sampled 100 examples = 43.0%
K-Fold 0 Step 5: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 0 Step 6: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 0 Step 7: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 8: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 0 Step 9: testing accuracy on random sampled 100 examples = 47.0%
K-Fold 0 Step 10: testing accuracy on random sampled 100 examples = 43.0%
K-Fold 0 Step 11: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 0 Step 12: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 0 Step 13: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 0 Step 14: testing accuracy on random sampled 100 examples = 45.0%
K-Fold 0 Step 15: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 16: testing accuracy on random sampled 100 examples = 47.0%
K-Fold 0 Step 17: testing accuracy on random sampled 100 examples = 48.0%
K-Fold 0 Step 18: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 19: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 20: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 21: testing accuracy on random sampled 100 examples = 46.0%
K-Fold 0 Step 22: testing accuracy on random sampled 100 examples = 45.0%
K-Fold 0 Step 23: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 0 Step 24: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 0 Step 25: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 0 Step 26: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 0 Step 27: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 28: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 0 Step 29: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 0 Step 30: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 0 Step 31: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 0 Step 32: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 33: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 0 Step 34: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 0 Step 35: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 0 Step 36: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 0 Step 37: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 0 Step 38: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 0 Step 39: testing accuracy on random sampled 100 examples = 52.0%
Final test accuracy = 18.5%
correct_number = 2123
total_number = 11470
1 K-Fold: training negative size 42856; training positve size 3022
1 K-Fold: tesing negative size 10714; test positve size 756
1 K-Fold: testing_number = 11470, total_number = 22940
K-Fold 1 Step 0: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 1 Step 1: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 1 Step 2: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 1 Step 3: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 1 Step 4: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 1 Step 5: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 1 Step 6: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 1 Step 7: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 1 Step 8: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 1 Step 9: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 1 Step 10: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 1 Step 11: testing accuracy on random sampled 100 examples = 46.0%
K-Fold 1 Step 12: testing accuracy on random sampled 100 examples = 43.0%
K-Fold 1 Step 13: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 1 Step 14: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 1 Step 15: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 1 Step 16: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 1 Step 17: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 1 Step 18: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 1 Step 19: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 1 Step 20: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 1 Step 21: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 1 Step 22: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 1 Step 23: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 1 Step 24: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 1 Step 25: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 1 Step 26: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 1 Step 27: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 1 Step 28: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 1 Step 29: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 1 Step 30: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 1 Step 31: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 1 Step 32: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 1 Step 33: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 1 Step 34: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 1 Step 35: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 1 Step 36: testing accuracy on random sampled 100 examples = 66.0%
K-Fold 1 Step 37: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 1 Step 38: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 1 Step 39: testing accuracy on random sampled 100 examples = 54.0%
Final test accuracy = 54.1%
correct_number = 8334
total_number = 22940
2 K-Fold: training negative size 42856; training positve size 3022
2 K-Fold: tesing negative size 10714; test positve size 756
2 K-Fold: testing_number = 11470, total_number = 34410
K-Fold 2 Step 0: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 2 Step 1: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 2 Step 2: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 2 Step 3: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 2 Step 4: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 2 Step 5: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 2 Step 6: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 2 Step 7: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 2 Step 8: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 2 Step 9: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 2 Step 10: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 2 Step 11: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 2 Step 12: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 2 Step 13: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 2 Step 14: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 2 Step 15: testing accuracy on random sampled 100 examples = 47.0%
K-Fold 2 Step 16: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 2 Step 17: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 2 Step 18: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 2 Step 19: testing accuracy on random sampled 100 examples = 48.0%
K-Fold 2 Step 20: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 2 Step 21: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 2 Step 22: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 2 Step 23: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 2 Step 24: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 2 Step 25: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 2 Step 26: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 2 Step 27: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 2 Step 28: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 2 Step 29: testing accuracy on random sampled 100 examples = 72.0%
K-Fold 2 Step 30: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 2 Step 31: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 2 Step 32: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 2 Step 33: testing accuracy on random sampled 100 examples = 48.0%
K-Fold 2 Step 34: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 2 Step 35: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 2 Step 36: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 2 Step 37: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 2 Step 38: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 2 Step 39: testing accuracy on random sampled 100 examples = 50.0%
Final test accuracy = 16.5%
correct_number = 10222
total_number = 34410
3 K-Fold: training negative size 42856; training positve size 3023
3 K-Fold: tesing negative size 10714; test positve size 755
3 K-Fold: testing_number = 11469, total_number = 45879
K-Fold 3 Step 0: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 3 Step 1: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 3 Step 2: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 3 Step 3: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 3 Step 4: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 3 Step 5: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 3 Step 6: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 3 Step 7: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 3 Step 8: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 3 Step 9: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 3 Step 10: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 3 Step 11: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 3 Step 12: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 3 Step 13: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 3 Step 14: testing accuracy on random sampled 100 examples = 67.0%
K-Fold 3 Step 15: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 3 Step 16: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 3 Step 17: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 3 Step 18: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 3 Step 19: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 3 Step 20: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 3 Step 21: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 3 Step 22: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 3 Step 23: testing accuracy on random sampled 100 examples = 64.0%
K-Fold 3 Step 24: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 3 Step 25: testing accuracy on random sampled 100 examples = 68.0%
K-Fold 3 Step 26: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 3 Step 27: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 3 Step 28: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 3 Step 29: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 3 Step 30: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 3 Step 31: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 3 Step 32: testing accuracy on random sampled 100 examples = 63.0%
K-Fold 3 Step 33: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 3 Step 34: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 3 Step 35: testing accuracy on random sampled 100 examples = 69.0%
K-Fold 3 Step 36: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 3 Step 37: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 3 Step 38: testing accuracy on random sampled 100 examples = 65.0%
K-Fold 3 Step 39: testing accuracy on random sampled 100 examples = 56.0%
Final test accuracy = 50.5%
correct_number = 16012
total_number = 45879
4 K-Fold: training negative size 42856; training positve size 3023
4 K-Fold: tesing negative size 10714; test positve size 755
4 K-Fold: testing_number = 11469, total_number = 57348
K-Fold 4 Step 0: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 1: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 4 Step 2: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 4 Step 3: testing accuracy on random sampled 100 examples = 48.0%
K-Fold 4 Step 4: testing accuracy on random sampled 100 examples = 50.0%
K-Fold 4 Step 5: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 4 Step 6: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 4 Step 7: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 4 Step 8: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 9: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 4 Step 10: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 4 Step 11: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 4 Step 12: testing accuracy on random sampled 100 examples = 49.0%
K-Fold 4 Step 13: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 4 Step 14: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 4 Step 15: testing accuracy on random sampled 100 examples = 52.0%
K-Fold 4 Step 16: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 4 Step 17: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 18: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 4 Step 19: testing accuracy on random sampled 100 examples = 56.0%
K-Fold 4 Step 20: testing accuracy on random sampled 100 examples = 62.0%
K-Fold 4 Step 21: testing accuracy on random sampled 100 examples = 51.0%
K-Fold 4 Step 22: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 4 Step 23: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 4 Step 24: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 4 Step 25: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 4 Step 26: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 4 Step 27: testing accuracy on random sampled 100 examples = 60.0%
K-Fold 4 Step 28: testing accuracy on random sampled 100 examples = 54.0%
K-Fold 4 Step 29: testing accuracy on random sampled 100 examples = 55.0%
K-Fold 4 Step 30: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 4 Step 31: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 4 Step 32: testing accuracy on random sampled 100 examples = 58.0%
K-Fold 4 Step 33: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 4 Step 34: testing accuracy on random sampled 100 examples = 53.0%
K-Fold 4 Step 35: testing accuracy on random sampled 100 examples = 59.0%
K-Fold 4 Step 36: testing accuracy on random sampled 100 examples = 57.0%
K-Fold 4 Step 37: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 4 Step 38: testing accuracy on random sampled 100 examples = 61.0%
K-Fold 4 Step 39: testing accuracy on random sampled 100 examples = 58.0%
Final test accuracy = 39.3%
correct_number = 20523
total_number = 57348
Final KFold test accuracy = 35.8%